<a href="https://colab.research.google.com/github/pgssanjana/micro_projects/blob/main/face_recognition_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install -q kaggle
from google.colab import files
files.upload()
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
# ! kaggle datasets list


Saving kaggle.json to kaggle (2).json
mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [4]:
!kaggle datasets download -d pramod722445/face-recognition-30


face-recognition-30.zip: Skipping, found more recently modified local copy (use --force to force download)


In [5]:
import zipfile
zf = "/content/face-recognition-30.zip"
target_dir = "/content/input/face-recognition-30/dataset"
zfile = zipfile.ZipFile(zf)
zfile.extractall(target_dir)

In [6]:
import numpy as np
import pandas as pd
import tensorflow as tf
import os
import cv2
import matplotlib.pyplot as plt
import matplotlib as mpl

# style your matplotlib
mpl.style.use("seaborn-darkgrid")
# run this block



In [7]:
from tqdm import tqdm

In [8]:
files=os.listdir("/content/input/face-recognition-30/dataset/dataset/")
files

['Johnny_Galeck',
 'Courteney_Cox',
 'Jennifer_Aniston',
 'aishwarya_rai',
 'hardik_pandya',
 'manoj_bajpayee',
 'Jim_Parsons',
 'angelina_jolie',
 'David_Schwimmer',
 'random_person',
 'scarlett_johansson',
 'sylvester_stallone',
 'mohamed_ali',
 'Lisa_Kudrow',
 'ronaldo',
 'Simon_Helberg',
 'Matthew_Perry',
 'Pankaj_Tripathi',
 'pewdiepie',
 'ROHIT_SHARMA',
 'dhoni',
 'Sachin_Tendulka',
 'suresh_raina',
 'brad_pitt',
 'bhuvan_bam',
 'messi',
 'virat_kohli',
 'arnold_schwarzenegger',
 'Kunal_Nayya',
 'Matt_LeBlanc']

In [9]:
index_2_name={}
name_2_index={}

In [10]:
image_array=[]  # it's a list later i will convert it to array
label_array=[]
path="/content/input/face-recognition-30/dataset/dataset/"
# loop through each sub-folder in train
for i in range(len(files)):
    # files in sub-folder
    file_sub=os.listdir(path+files[i])
    index_2_name[i]=files[i]
    name_2_index[files[i]]=i

    for k in tqdm(range(len(file_sub))):
        try:
            img=cv2.imread(path+files[i]+"/"+file_sub[k])
            img=cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
            img=cv2.resize(img,(96,96))
            image_array.append(img)
            label_array.append(i)
        except:
            pass

100%|██████████| 449/449 [00:01<00:00, 373.66it/s]


In [11]:
print(index_2_name)
print()
print(name_2_index)


{0: 'Johnny_Galeck', 1: 'Courteney_Cox', 2: 'Jennifer_Aniston', 3: 'aishwarya_rai', 4: 'hardik_pandya', 5: 'manoj_bajpayee', 6: 'Jim_Parsons', 7: 'angelina_jolie', 8: 'David_Schwimmer', 9: 'random_person', 10: 'scarlett_johansson', 11: 'sylvester_stallone', 12: 'mohamed_ali', 13: 'Lisa_Kudrow', 14: 'ronaldo', 15: 'Simon_Helberg', 16: 'Matthew_Perry', 17: 'Pankaj_Tripathi', 18: 'pewdiepie', 19: 'ROHIT_SHARMA', 20: 'dhoni', 21: 'Sachin_Tendulka', 22: 'suresh_raina', 23: 'brad_pitt', 24: 'bhuvan_bam', 25: 'messi', 26: 'virat_kohli', 27: 'arnold_schwarzenegger', 28: 'Kunal_Nayya', 29: 'Matt_LeBlanc'}

{'Johnny_Galeck': 0, 'Courteney_Cox': 1, 'Jennifer_Aniston': 2, 'aishwarya_rai': 3, 'hardik_pandya': 4, 'manoj_bajpayee': 5, 'Jim_Parsons': 6, 'angelina_jolie': 7, 'David_Schwimmer': 8, 'random_person': 9, 'scarlett_johansson': 10, 'sylvester_stallone': 11, 'mohamed_ali': 12, 'Lisa_Kudrow': 13, 'ronaldo': 14, 'Simon_Helberg': 15, 'Matthew_Perry': 16, 'Pankaj_Tripathi': 17, 'pewdiepie': 18, 'R

In [12]:
import gc
gc.collect()

100

In [13]:
image_array=np.array(image_array)/255.0
label_array=np.array(label_array)


In [14]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test=train_test_split(image_array,label_array,test_size=0.15)

In [15]:
from keras import layers,callbacks,utils,applications,optimizers
from keras.models import Sequential,Model,load_model

In [16]:
len(files)


30

In [17]:
import tensorflow as tf
import keras
from tensorflow.keras.preprocessing import image_dataset_from_directory
from keras.preprocessing.image import ImageDataGenerator
# from keras.applications import MobileNet
from keras.applications.mobilenet import preprocess_input

In [18]:
from tensorflow.keras.applications.resnet50 import ResNet50

In [19]:
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2

In [20]:

model=Sequential()
# I will use MobileNetV2 as an pretrained model 
pretrained_model=MobileNetV2(input_shape=(96,96,3),include_top=False,
                                         weights="imagenet")
model.add(pretrained_model)
model.add(layers.GlobalAveragePooling2D())
# add dropout to increase accuracy by not overfitting
model.add(layers.Dropout(0.3))
# add dense layer as final output
model.add(layers.Dense(1))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 mobilenetv2_1.00_96 (Functi  (None, 3, 3, 1280)       2257984   
 onal)                                                           
                                                                 
 global_average_pooling2d (G  (None, 1280)             0         
 lobalAveragePooling2D)                                          
                                                                 
 dropout (Dropout)           (None, 1280)              0         
                                                                 
 dense (Dense)               (None, 1)                 1281      
                                                                 
Total params: 2,259,265
Trainable params: 2,225,153
Non-trainable params: 34,112
_________________________________________________________________


In [21]:
model.compile(optimizer="adam",loss="mean_squared_error",metrics=["mae"])



In [22]:

ckp_path="trained_model/model"
model_checkpoint=tf.keras.callbacks.ModelCheckpoint(filepath=ckp_path,
                                                   monitor="val_mae",
                                                   mode="auto",
                                                   save_best_only=True,
                                                   save_weights_only=True)


In [23]:
# create a lr reducer which decrease learning rate when accuarcy does not increase
reduce_lr=tf.keras.callbacks.ReduceLROnPlateau(factor=0.9,monitor="val_mae",
                                             mode="auto",cooldown=0,
                                             patience=5,verbose=1,min_lr=1e-6)
# patience : wait till 5 epoch
# verbose : show accuracy every 1 epoch
# min_lr=minimum learning rate

In [24]:
# import tensorflow as tf
# my_callbacks = [
#     tf.keras.callbacks.EarlyStopping(patience=2),
#     tf.keras.callbacks.ModelCheckpoint(filepath='model.{epoch:02d}-{val_loss:.2f}.h5',
#                               monitor='val_loss',
#                              verbose=1, 
#                              save_best_only=True),
#     tf.keras.callbacks.TensorBoard(log_dir='./logs'),
# ]

In [25]:
EPOCHS=300
BATCH_SIZE=64

history=model.fit(X_train,
                 Y_train,
                 validation_data=(X_test,Y_test),
                 batch_size=BATCH_SIZE,
                 epochs=EPOCHS,
                 callbacks=[model_checkpoint,reduce_lr],
                  steps_per_epoch=212
                  

                 )

Epoch 1/300
212/212 [==============================] - 35s 94ms/step - loss: 35.5083 - mae: 4.2867 - val_loss: 353.4930 - val_mae: 17.5709 - lr: 0.0010
Epoch 2/300
212/212 [==============================] - 17s 80ms/step - loss: 14.4682 - mae: 2.6398 - val_loss: 277.3805 - val_mae: 15.5134 - lr: 0.0010
Epoch 3/300
212/212 [==============================] - 17s 82ms/step - loss: 9.9027 - mae: 2.0931 - val_loss: 244.5457 - val_mae: 14.0846 - lr: 0.0010
Epoch 4/300
212/212 [==============================] - 17s 81ms/step - loss: 6.7342 - mae: 1.6970 - val_loss: 261.1548 - val_mae: 14.4133 - lr: 0.0010
Epoch 5/300
212/212 [==============================] - 17s 82ms/step - loss: 8.3989 - mae: 1.8974 - val_loss: 295.8155 - val_mae: 15.7423 - lr: 0.0010
Epoch 6/300
212/212 [==============================] - 18s 84ms/step - loss: 7.7216 - mae: 1.7868 - val_loss: 323.1360 - val_mae: 16.6243 - lr: 0.0010
Epoch 7/300
212/212 [==============================] - 19s 88ms/step - loss: 5.6005 - mae: 1

In [26]:
prediction_val=model.predict(X_test,batch_size=64)
prediction_val[:20]

array([[25.06678   ],
       [28.896982  ],
       [ 4.7628436 ],
       [14.1008005 ],
       [20.367718  ],
       [27.802975  ],
       [ 6.89298   ],
       [22.013563  ],
       [28.91738   ],
       [ 9.054654  ],
       [20.126543  ],
       [16.924988  ],
       [18.190886  ],
       [ 8.913317  ],
       [15.081546  ],
       [ 6.7801213 ],
       [13.7944145 ],
       [ 1.0407615 ],
       [-0.07920676],
       [22.609425  ]], dtype=float32)

In [27]:
model.load_weights(ckp_path)

In [28]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the model.
with open('model_new.tflite', 'wb') as f:
  f.write(tflite_model)

INFO:tensorflow:Assets written to: /tmp/tmp91qx14zp/assets


INFO:tensorflow:Assets written to: /tmp/tmp91qx14zp/assets


In [32]:
TF_LITE_MODEL_FILE_NAME = 'model_new.tflite'

In [33]:
interpreter = tf.lite.Interpreter(model_path = TF_LITE_MODEL_FILE_NAME)
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
print("Input Shape:", input_details[0]['shape'])
print("Input Type:", input_details[0]['dtype'])
print("Output Shape:", output_details[0]['shape'])
print("Output Type:", output_details[0]['dtype'])

Input Shape: [ 1 96 96  3]
Input Type: <class 'numpy.float32'>
Output Shape: [1 1]
Output Type: <class 'numpy.float32'>


In [34]:
X_test.shape


(2384, 96, 96, 3)

In [35]:
import numpy as np
X_test = np.zeros( (2384, 96, 96, 3) )
X_test = X_test[:3, :, ]
print(X_test.shape)

(3, 96, 96, 3)


In [36]:


test_image = np.expand_dims(X_test, axis=0)
test_image=np.array(X_test,dtype=np.float32)


In [37]:
print(interpreter.get_input_details())

[{'name': 'serving_default_mobilenetv2_1.00_96_input:0', 'index': 0, 'shape': array([ 1, 96, 96,  3], dtype=int32), 'shape_signature': array([-1, 96, 96,  3], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]


In [31]:
import pickle
filename='pred_model'
pickle.dump(model,open(filename,'wb'))

INFO:tensorflow:Assets written to: ram://dc23d7db-856c-44af-a905-566212610d24/assets


INFO:tensorflow:Assets written to: ram://dc23d7db-856c-44af-a905-566212610d24/assets


In [39]:
loaded_model=pickle.load(open(filename,'rb'))
loaded_model.predict(X_test)

array([[9.465995],
       [9.465995],
       [9.465995]], dtype=float32)

In [40]:
with open('pickle_model','wb') as f:
  pickle.dump(model,f )

INFO:tensorflow:Assets written to: ram://9bf5d5ed-3d5a-456d-a582-36b8fd67ce0b/assets


INFO:tensorflow:Assets written to: ram://9bf5d5ed-3d5a-456d-a582-36b8fd67ce0b/assets


In [41]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the model.
with open('model_new_facerec.tflite', 'wb') as f:
  f.write(tflite_model)

INFO:tensorflow:Assets written to: /tmp/tmp0g158y80/assets


INFO:tensorflow:Assets written to: /tmp/tmp0g158y80/assets


In [85]:
from keras.preprocessing import image
img = image.load_img('/content/test.jpeg',target_size=(96,96))

x = image.img_to_array(img)

x=np.expand_dims(x,axis=0)

x=preprocess_input(x)

In [86]:
pred = model.predict(x)
pred
num=round(pred[0][0])
print(index_2_name[num])

David_Schwimmer
